In [21]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc
from sklearn.datasets import load_breast_cancer

import matplotlib.pyplot as plt
import seaborn as sns

from aggmap import AggMap, AggModel

np.random.seed(666) #just for reaptable results

def prc_auc_score(y_true, y_score):
    precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
    auc = calculate_auc(recall, precision)
    return auc

In [22]:
data = load_breast_cancer()
dfx = pd.DataFrame(data.data, columns= data.feature_names)
y = pd.Series(data.target).map({0:'malignant', 1:'benign'})
dfy = pd.get_dummies(y) 
X = dfx.values.astype(float)
Y = dfy.values.astype(float)

In [23]:
mp = AggMap(dfx, metric = 'correlation')
mp.fit(cluster_channels = 5, verbose = 0)

2020-08-18 14:08:36,002 - INFO - [bidd-aggmap] - Calculating distance ...
2020-08-18 14:08:36,004 - INFO - [bidd-aggmap] - the number of process is 16


100%|##########| 30/30 [00:00<00:00, 1049.24it/s]


2020-08-18 14:08:37,652 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2020-08-18 14:08:37,767 - INFO - [bidd-aggmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2020-08-18 14:08:37,786 - INFO - [bidd-aggmap] - Finished


## 10-fold cross validation and repeat 5 times 

In [24]:
outer_fold = 10

run_all = []
for repeat_seed in [8, 16, 32, 64, 128]: #5 repeats random seeds
    
    outer = KFold(n_splits = outer_fold, shuffle = True, random_state = repeat_seed)
    outer_idx = outer.split(range(len(Y)), Y[:,0])

    for i, idx in enumerate(outer_idx):
        
        fold_num = "fold_%s" % str(i).zfill(2) 
        
        train_idx, test_idx = idx
        
        testY = Y[test_idx]
        testx = X[test_idx]
        
        trainx = X[train_idx]
        trainY = Y[train_idx]

        trainX = mp.batch_transform(trainx, scale_method = 'standard')
        testX = mp.batch_transform(testx, scale_method = 'standard')
        
        print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))
        
        clf = AggModel.MultiClassEstimator(epochs = 200,  verbose = 0)
        clf.fit(trainX, trainY)

        y_true = testY[:,0]
        y_pred = clf.predict(testX)[:,0]
        y_score = clf.predict_proba(testX)[:,0]

        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        
        acc = (tp + tn) / sum([tn, fp, fn, tp])
        
        sensitivity = tp / sum([tp, fn])
        specificity = tn / sum([tn, fp])
        
        prc_auc = prc_auc_score(y_true, y_score)
        roc_auc = roc_auc_score(y_true, y_score)
        
        precision = tp / sum([tp, fp])
        recall =  tp / sum([tp, fn]) #equals to sensitivity
        
        
        res = {'fold': fold_num,
               'repeat_seed':repeat_seed,
               
               'accuracy':acc, 
               
               'prc_auc':prc_auc, 
               'roc_auc':roc_auc,

               'sensitivity': sensitivity, 
               'specificity': specificity,
               
               'precision':precision,
               'recall':recall,
              }
        
        run_all.append(res)

100%|##########| 57/57 [00:00<00:00, 1326.05it/s]



 input train and test X shape is (512, 6, 5, 5), (57, 6, 5, 5) 
MultiClassEstimator(batch_size=128, conv1_kernel_size=11, dense_avf='relu',
                    dense_layers=[128], epochs=200, lr=0.0001, metric='ROC',
                    monitor='val_loss', name='AggMap MultiClass Estimator',
                    patience=10000, random_state=32, verbose=0)


100%|##########| 57/57 [00:00<00:00, 3568.50it/s]



 input train and test X shape is (512, 6, 5, 5), (57, 6, 5, 5) 
MultiClassEstimator(batch_size=128, conv1_kernel_size=11, dense_avf='relu',
                    dense_layers=[128], epochs=200, lr=0.0001, metric='ROC',
                    monitor='val_loss', name='AggMap MultiClass Estimator',
                    patience=10000, random_state=32, verbose=0)


100%|##########| 57/57 [00:00<00:00, 3599.94it/s]


...


 input train and test X shape is (512, 6, 5, 5), (57, 6, 5, 5) 
MultiClassEstimator(batch_size=128, conv1_kernel_size=11, dense_avf='relu',
                    dense_layers=[128], epochs=200, lr=0.0001, metric='ROC',
                    monitor='val_loss', name='AggMap MultiClass Estimator',
                    patience=10000, random_state=32, verbose=0)


100%|##########| 57/57 [00:00<00:00, 560.65it/s]



 input train and test X shape is (512, 6, 5, 5), (57, 6, 5, 5) 
MultiClassEstimator(batch_size=128, conv1_kernel_size=11, dense_avf='relu',
                    dense_layers=[128], epochs=200, lr=0.0001, metric='ROC',
                    monitor='val_loss', name='AggMap MultiClass Estimator',
                    patience=10000, random_state=32, verbose=0)


100%|##########| 56/56 [00:00<00:00, 1102.69it/s]



 input train and test X shape is (513, 6, 5, 5), (56, 6, 5, 5) 
MultiClassEstimator(batch_size=128, conv1_kernel_size=11, dense_avf='relu',
                    dense_layers=[128], epochs=200, lr=0.0001, metric='ROC',
                    monitor='val_loss', name='AggMap MultiClass Estimator',
                    patience=10000, random_state=32, verbose=0)


In [25]:
clf._model.count_params()

323138

In [26]:
df = pd.DataFrame(run_all)
df.to_excel('results.xlsx')
df

,fold,repeat_seed,accuracy,prc_auc,roc_auc,sensitivity,specificity,precision,recall
0,fold_00,8,0.982456,0.998907,0.998765,1.000000,0.962963,0.967742,1.000000
1,fold_01,8,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,fold_02,8,0.947368,0.996187,0.993243,0.945946,0.950000,0.972222,0.945946
3,fold_03,8,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,fold_04,8,0.964912,0.991985,0.988491,1.000000,0.913043,0.944444,1.000000
5,fold_05,8,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6,fold_06,8,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
7,fold_07,8,0.964912,0.981654,0.979798,0.969697,0.958333,0.969697,0.969697
8,fold_08,8,0.947368,1.000000,1.000000,1.000000,0.850000,0.925000,1.000000
9,fold_09,8,0.946429,1.000000,1.000000,0.933333,1.000000,1.000000,0.933333


In [30]:
results = df.groupby(['repeat_seed']).apply(np.mean)
results

,repeat_seed,accuracy,prc_auc,roc_auc,sensitivity,specificity,precision,recall
repeat_seed,,,,,,,,
8,8.0,0.975345,0.996873,0.996030,0.984898,0.963434,0.977911,0.984898
16,16.0,0.973622,0.997132,0.995511,0.980651,0.962623,0.977630,0.980651
32,32.0,0.975439,0.996618,0.995478,0.982952,0.964229,0.977690,0.982952
64,64.0,0.971930,0.996679,0.994228,0.985473,0.943389,0.972402,0.985473
128,128.0,0.971898,0.995888,0.993986,0.983558,0.952724,0.974088,0.983558


In [31]:
results.mean().round(3)

repeat_seed    49.600
accuracy        0.974
prc_auc         0.997
roc_auc         0.995
sensitivity     0.984
specificity     0.957
precision       0.976
recall          0.984
dtype: float64

In [32]:
results.std().round(3)

repeat_seed    48.793
accuracy        0.002
prc_auc         0.000
roc_auc         0.001
sensitivity     0.002
specificity     0.009
precision       0.003
recall          0.002
dtype: float64